[View in Colaboratory](https://colab.research.google.com/github/akamloo/ML-Practice/blob/master/Spam_Mail.ipynb)

In [0]:
import os
from collections import Counter
import numpy as np
def create_dictionary(mail_dir):
  all_words = []
  mails = [os.path.join(mail_dir,f) for f in os.listdir(mail_dir)]
  for mail in mails:
    with open(mail) as m:
      for line in m:
        words = line.split()
        all_words += words
  dictionary = Counter(all_words)
  list_to_remove = list(dictionary)
  for item in list_to_remove:
    if item.isalpha() == False:
      del dictionary[item]
    elif len(item) == 1:
      del dictionary[item]
  dictionary = dictionary.most_common(3000)
  return dictionary

def extract_features(mail_dir, dictionary):
  files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
  features_matrix = np.zeros((len(files),3000))
  file_labels = np.zeros(len(files))
  count = 0
  doc_id = 0
  for file in files:
    with open(file) as fi:
      for i, line in enumerate(fi):
        if i == 2:
          words = line.split()
          for word in words:
            word_id = 0
            for i, d in enumerate(dictionary):
              if d[0] == word:
                word_id = i
                features_matrix[doc_id, word_id] = words.count(word)
      file_labels[doc_id] = 0
      file_path_tokens = file.split('/')
      last_token = file_path_tokens[len(file_path_tokens)-1]
      if last_token.startswith("spmsg"):
        file_labels[doc_id] = 1
        count += 1
      doc_id += 1
  return features_matrix, file_labels

In [25]:
!ls
os.getcwd()

drive  sample_data  spam_mails


'/content'

In [0]:
from sklearn import svm
from sklearn.metrics import accuracy_score

training_dir = os.getcwd()+'/spam_mails/train'
test_dir = os.getcwd()+'/spam_mails/test'

In [0]:
training_dir
dictionary = create_dictionary(training_dir)

In [44]:
print("Reading and preprocessing email from files ...")
feature_matrix, file_label = extract_features(training_dir, dictionary)
model = svm.SVC()
print("Training model using SVM ...")
model.fit(feature_matrix, file_label)

Reading and preprocessing email from files ...
Training model using SVM ...


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [46]:
feature_matrix_test, file_label_test = extract_features(test_dir, dictionary)
predicted_labels = model.predict(feature_matrix_test)

print("Finished classification... Accuracy score : ")
print(accuracy_score(file_label_test, predicted_labels))

Finished classification... Accuracy score : 
0.8153846153846154


In [71]:
tuned_model = svm.SVC(kernel='rbf', C=100, gamma=0.00012)
tuned_model.fit(feature_matrix, file_label)

predicted_labels = tuned_model.predict(feature_matrix_test)
print("Finished classification of tuned model... Accuracy score : ")
print(accuracy_score(file_label_test, predicted_labels))

Finished classification of tuned model... Accuracy score : 
0.9807692307692307
